In [1]:
import dill as pickle
import sk8s
from flask import Flask, request
import json
import queue
import threading

import base64
import sys


def worker_thread(jobs, results):
    while True:
        name, func = jobs.get()
        results[name] = func()

    
class LambdaWorker:
    def __init__(self):
        self.app = Flask(__name__)

        self.queue = queue.Queue()
        self.results = dict()
        self.worker = threading.Thread(target=worker_thread,
                                       args=[self.queue, self.results])

        self.app.add_url_rule('/enqueue', 'enqueue', self.enqueue, methods=["POST"])
        self.app.add_url_rule('/check/<task_id>', 'check', self.check, methods=["GET"])
        self.app.add_url_rule('/echo/<msg>', 'echo', self.echo, methods=["GET"])
        self.app.add_url_rule('/shutdown', 'shutdown', self.shutdown, methods=["GET"])

    def enqueue(self):
        func = sk8s.deserialize_func(json.loads(request.data))
        task_id = "lambda-{s}".format(s=sk8s.random_string(5))
        self.queue.put((task_id, func))
        return task_id

    def check(self, task_id):
        return json.dumps(self.results.get(task_id, None))

    def echo(self, msg):
        return json.dumps(msg)

    def shutdown(self):
        func = request.environ.get('werkzeug.server.shutdown')
        self.running = False
        if func is None:
            raise RuntimeError('Not running with the Werkzeug Server')
        func()
    
    def run(self):
        self.running = True
        self.worker.start()
        return self.app.run(host="0.0.0.0")


In [2]:
worker = LambdaWorker()


#import threading
#thread = threading.Thread(target=pool.run, args=[])
#thread.start()